In [1]:
# imports
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

10000 - arts and entertainment

12009 - education
12004 - community center
12080 - library
19046 - metro station
15014 - hospital


13032 - cafes, coffee, tea houses
13029 - brewery
13003 - bar
13065 - restaurant

16000 - landmarks and outdoors
18000 - sports and rec

17057 - retail food/drink

17029 - convenience store
17035 - drug store



In [56]:
bike_df = pd.read_csv('bike_df')
bike_df.head()

,Station_name,Latitude,Longitude,Bikes,Coord_String
0,10th & Cambie,49.262487,-123.114397,35,49.262487%2C-123.114397
1,Yaletown-Roundhouse Station,49.274566,-123.121817,16,49.274566%2C-123.121817
2,Dunsmuir & Beatty,49.279764,-123.110154,26,49.279764%2C-123.110154
3,12th & Yukon (City Hall),49.260599,-123.113504,16,49.260599%2C-123.113504
4,8th & Ash,49.264215,-123.117772,16,49.264215%2C-123.117772


In [57]:
testbike_df = bike_df.head(10)
testbike_df

,Station_name,Latitude,Longitude,Bikes,Coord_String
0,10th & Cambie,49.262487,-123.114397,35,49.262487%2C-123.114397
1,Yaletown-Roundhouse Station,49.274566,-123.121817,16,49.274566%2C-123.121817
2,Dunsmuir & Beatty,49.279764,-123.110154,26,49.279764%2C-123.110154
3,12th & Yukon (City Hall),49.260599,-123.113504,16,49.260599%2C-123.113504
4,8th & Ash,49.264215,-123.117772,16,49.264215%2C-123.117772
5,Spyglass & Seawall,49.270877,-123.115103,14,49.270877%2C-123.115103
6,Stamps Landing,49.268930,-123.118130,14,49.26893%2C-123.11813
7,Ontario & Seawall,49.271721,-123.104207,20,49.271721%2C-123.104207
8,Canada Place,49.287214,-123.112870,26,49.287214%2C-123.11287
9,Granville & Georgia,49.282409,-123.118541,20,49.282409%2C-123.118541


In [99]:
fs_api_key = os.environ['FOURSQUARE_KEY_2']
foursq_dict = {'Category':[], 'Name': [], 'Assc Bike Lat': [], 'Assc Bike Long': [] }
category_map = {0: 'Arts & Entertainment', 1: 'Schools', 2: 'Municipal', 3: 'Cafes', 4: 'Restaurants', 5: 'Bars',
                6: 'Landmarks, Parks & Sports', 7: 'Retail (Food/Bev)', 8: 'Drug and Convenience Stores', 9: 'Retail (Clothing)', 10: 'Retail (Other goods)'}


# target POI categories from FourSquare
arts_ent = '10000'
schools = '12014%2C12059%2C' 
municipal = '12004%2C12080%2C19046%2C15014'
cafes = '13032'
restaurants = '13065'
bars = '13029%2C13003'
outdoors = '16000%2C18000'
retail_food = '17057'
retail_drug_conv = '17029%2C17035'
retail_clothes = '17043'
retail_goods = '17003%2C17016%2C17018%2C17022%2C17023%2C17033%2C17114'

for location in bike_df['Coord_String'].values:
    # For current bike station location, loop through categories and send request for each category
    categories = [arts_ent, schools, municipal, cafes, restaurants, bars, outdoors, retail_food, retail_drug_conv, retail_clothes, retail_goods]
    responses = []
    for category in categories:
        url = "https://api.foursquare.com/v3/places/search?radius=300&limit=50&ll=" + location + '&categories=' + category

        headers = {"accept": "application/json",
                'Authorization': fs_api_key}

        response = requests.get(url, headers=headers)
        responses.append(response)
        print(location, category, response.status_code)

    # loop through responses to pull out results.
    for response in responses:
        for place in response.json()['results']:
            # pull name
            name = place['name']
            foursq_dict['Name'].append(name)

            # find coordinates of associated bike station
            station_lat = response.json()['context']['geo_bounds']['circle']['center']['latitude']
            station_long = response.json()['context']['geo_bounds']['circle']['center']['longitude']
            foursq_dict['Assc Bike Lat'].append(station_lat)
            foursq_dict['Assc Bike Long'].append(station_long)

            # find category
            category = category_map[responses.index(response)]
            foursq_dict['Category'].append(category)


49.262487%2C-123.114397 10000 200
49.262487%2C-123.114397 12014%2C12059%2C 200
49.262487%2C-123.114397 12004%2C12080%2C19046%2C15014 200
49.262487%2C-123.114397 13032 200
49.262487%2C-123.114397 13065 200
49.262487%2C-123.114397 13029%2C13003 200
49.262487%2C-123.114397 16000%2C18000 200
49.262487%2C-123.114397 17057 200
49.262487%2C-123.114397 17029%2C17035 200
49.262487%2C-123.114397 17043 200
49.262487%2C-123.114397 17003%2C17016%2C17018%2C17022%2C17023%2C17033%2C17114 200
49.274566%2C-123.121817 10000 200
49.274566%2C-123.121817 12014%2C12059%2C 200
49.274566%2C-123.121817 12004%2C12080%2C19046%2C15014 200
49.274566%2C-123.121817 13032 200
49.274566%2C-123.121817 13065 200
49.274566%2C-123.121817 13029%2C13003 200
49.274566%2C-123.121817 16000%2C18000 200
49.274566%2C-123.121817 17057 200
49.274566%2C-123.121817 17029%2C17035 200
49.274566%2C-123.121817 17043 200
49.274566%2C-123.121817 17003%2C17016%2C17018%2C17022%2C17023%2C17033%2C17114 200
49.279764%2C-123.110154 10000 200
49.2

Put your parsed results into a DataFrame

In [100]:
foursq_df = pd.DataFrame(foursq_dict)

In [102]:
foursq_df.to_csv('foursquare_df', index=False)

##### Data Cleaning

* drop duplicate rows
* combine bar, restaurant and cafe categories and drop duplicates
* combine retail categories and drop duplicates




In [68]:
foursq_df = pd.read_csv('foursquare_df')
foursq_df.shape

(19768, 4)

In [70]:
foursq_df[foursq_df.duplicated(keep=False)]

,Category,Name,Assc Bike Lat,Assc Bike Long
34,Restaurants,A&W Restaurant,49.262487,-123.114397
35,Restaurants,A&W Restaurant,49.262487,-123.114397
145,Retail (Other goods),Canadian Tire,49.262487,-123.114397
152,Retail (Other goods),Canadian Tire,49.262487,-123.114397
181,Cafes,Starbucks,49.274566,-123.121817
...,...,...,...,...
19743,Drug and Convenience Stores,Shoppers Drug Mart,49.277527,-123.129464
19744,Drug and Convenience Stores,Guardians,49.277527,-123.129464
19745,Drug and Convenience Stores,Pharmachoice,49.277527,-123.129464
19747,Drug and Convenience Stores,Pharmachoice,49.277527,-123.129464


In [69]:
foursq_df_dupes = foursq_df.drop_duplicates()
foursq_df_dupes.shape

(19304, 4)

In [71]:
foursq_df=foursq_df_dupes
del foursq_df_dupes

In [80]:
# Collapse Food/Drink categories
bar_filter = foursq_df['Category'] == 'Bars'
rest_filter = foursq_df['Category'] == 'Restaurants'
cafe_filter = foursq_df['Category'] == 'Cafes'

foursq_df['Category'] = np.where((bar_filter | rest_filter | cafe_filter), 'Bars, Restaurants & Cafes', foursq_df['Category'])

In [82]:
foursq_df_dupes = foursq_df.copy()
foursq_df_dupes.drop_duplicates(inplace=True)
foursq_df_dupes.shape

(18055, 4)

In [83]:
foursq_df = foursq_df_dupes.copy()
del foursq_df_dupes

In [85]:
# Collapse Retail categories
retail_filter1 = foursq_df['Category'] == 'Retail (Food/Bev)'
retail_filter2 = foursq_df['Category'] == 'Drug and Convenience Stores'
retail_filter3 = foursq_df['Category'] == 'Retail (Clothing)'
retail_filter4 = foursq_df['Category'] == 'Retail (Other goods)'

foursq_df['Category'] = np.where((retail_filter1 | retail_filter2 | retail_filter3 | retail_filter4), 'Retail', foursq_df['Category'])
foursq_df_dupes = foursq_df.copy()
foursq_df_dupes.drop_duplicates(inplace=True)
foursq_df_dupes.shape

(18040, 4)

In [86]:
foursq_df = foursq_df_dupes.copy()
del foursq_df_dupes

In [90]:
foursq_df.to_csv('foursquare_df_clean', index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

active
arts
education
breweries
hospitals
metrostations
bars
communitycenters
landmarks
libraries
restaurants
shopping

In [64]:

yelp_key = os.environ['YELP_API_KEY']
yelp_dict = {'Category':[], 'Name': [], 'Rating': [], 'Reviews': [], 'Assc Bike Lat': [], 'Assc Bike Long': [] }
category_map = {0: 'Arts & Entertainment', 1: 'Schools', 2: 'Municipal', 3: 'Cafes', 4: 'Restaurants', 5: 'Bars',
                6: 'Landmarks, Parks & Sports', 7: 'Retail (Food/Bev)', 8: 'Drug and Convenience Stores', 9: 'Retail (Clothing)', 10: 'Retail (Other goods)'}


# target POI categories from Yelp
arts_ent = '&categories=arcades&categories=galleries&categories=gardens&categories=cabaret&categories=casinos&categories=movietheaters&categories=museums&categories=musicvenues&categories=theater&categories=virtualrealitycenters'
schools = '&categories=collegeuniv&categories=highschools' 
municipal = '&categories=hospitals&categories=metrostations&categories=communitycenters&categories=libraries'
cafes = '&categories=cafes'
restaurants = '&categories=restaurants'
bars = '&categories=bars&categories=breweries'
outdoors = '&categories=active&categories=landmarks'
retail_food = '&categories=beer_and_wine&categories=grocery&categories=farmersmarket'
retail_drug_conv = '&categories=drugstores&categories=convenience'
retail_clothes = '&categories=fashion'
retail_goods = '&categories=shoppingcenters&categories=artsandcrafts&categories=media&categories=electronics'

for lat, long in bike_df[['Latitude', 'Longitude']].values:
    latitude = str(lat)
    longitude = str(long)

    categories = [arts_ent, schools, municipal, cafes, restaurants, bars, outdoors, retail_food, retail_drug_conv, retail_clothes, retail_goods]
    responses = []

    for category in categories:
        url = "https://api.yelp.com/v3/businesses/search?radius=300&limit=50&latitude=" + latitude + '&longitude=' + longitude + category

        headers = {"accept": "application/json", 'Authorization': 'Bearer '+ yelp_key}

        response = requests.get(url, headers=headers)
        print(latitude, longitude, category, response.status_code)
        responses.append(response)
    
    for response in responses:
        for place in response.json()['businesses']:
            name = place['name']
            yelp_dict['Name'].append(name)
            
            rating = place['rating']
            yelp_dict['Rating'].append(rating)
            
            reviews = place['review_count']
            yelp_dict['Reviews'].append(reviews)

            station_lat = response.json()['region']['center']['latitude']
            station_long = response.json()['region']['center']['longitude']
            yelp_dict['Assc Bike Lat'].append(station_lat)
            yelp_dict['Assc Bike Long'].append(station_long)

            category = category_map[responses.index(response)]
            yelp_dict['Category'].append(category)

49.262487 -123.114397 &categories=arcades&categories=galleries&categories=gardens&categories=cabaret&categories=casinos&categories=movietheaters&categories=museums&categories=musicvenues&categories=theater&categories=virtualrealitycenters 200
49.262487 -123.114397 &categories=collegeuniv&categories=highschools 200
49.262487 -123.114397 &categories=hospitals&categories=metrostations&categories=communitycenters&categories=libraries 200
49.262487 -123.114397 &categories=cafes 200
49.262487 -123.114397 &categories=restaurants 200
49.262487 -123.114397 &categories=bars&categories=breweries 200
49.262487 -123.114397 &categories=active&categories=landmarks 200
49.262487 -123.114397 &categories=beer_and_wine&categories=grocery&categories=farmersmarket 200
49.262487 -123.114397 &categories=drugstores&categories=convenience 200
49.262487 -123.114397 &categories=fashion 200
49.262487 -123.114397 &categories=shoppingcenters&categories=artsandcrafts&categories=media&categories=electronics 200
49.27

Put your parsed results into a DataFrame

In [65]:
yelp_df=pd.DataFrame(yelp_dict)
yelp_df.head()

,Category,Name,Rating,Reviews,Assc Bike Lat,Assc Bike Long
0,Arts & Entertainment,Aperture Coffee Bar,4.0,122,49.262487,-123.114397
1,Arts & Entertainment,Evolve Virtual Reality,4.0,13,49.262487,-123.114397
2,Arts & Entertainment,Hidden Wonders,5.0,12,49.262487,-123.114397
3,Arts & Entertainment,V & G Entertainment,5.0,10,49.262487,-123.114397
4,Arts & Entertainment,"Under the Piano - Sound Spa for Body, Mind & Soul",5.0,9,49.262487,-123.114397


In [66]:
yelp_df.shape

(20174, 6)

In [72]:
yelp_df.to_csv('yelp_df', index=False)

##### Data Cleaning

* drop duplicate rows
* combine bar, restaurant and cafe categories and drop duplicates
* combine retail categories and drop duplicates


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating